In [142]:
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from datetime import timedelta

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error

In [143]:
route = pd.read_csv("/content/drive/MyDrive/Dataset/upa/preprocess/T-001_route2.csv",parse_dates=["Date from","Date to"])
env = pd.read_csv("/content/drive/MyDrive/Dataset/upa/T-001_env.csv",parse_dates=["Date"])
route = route.iloc[:93,:]
env.drop("No.",axis=1,inplace=True)
env.set_index("Date",inplace=True)

In [144]:
route.tail()

,Date from,Date to,days,Port from,Port to,Distance,T from,T to,Loaing Status,Mega-Ton-Mile,Cargo Carried,DFOC,Wave (per Distance),Acc. Fatigue Index,Wind (per Distance),Avg. Wave Heigh,Avg. Speed,Avg. Win Speed
88,2018-11-29 20:28:00,NaT,NaN,CA0550,BR0171,9359,12,9,NaN,0,0,0.0,0.9,168.4,4.7,0.9,NaN,4.6
89,2018-12-20 16:36:00,2018-12-22 13:39:00,1.9,BR0171,unown,47,9,9,NaN,0,0,0.0,0.4,0.3,1.9,0.3,8.2,1.8
90,2018-12-22 13:39:00,2018-12-26 21:20:00,4.3,unown,BR0145,189,9,9,NaN,0,0,0.0,0.5,1.3,1.2,0.4,12.8,1.1
91,2018-12-26 21:20:00,2018-12-29 13:32:00,2.7,BR0145,BR0010,19,9,16,NaN,254.266,75096,0.0,0.5,1.3,1.2,0.4,7.7,1.1
92,2018-01-13 01:22:00,2018-12-29 13:32:00,350.5,NaN,NaN,57095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.1,12.2,NaN


In [145]:
env.tail()

,Vessel Latitude,Vessel Longitude,Vessel Speed (Kts),Vessel Heading,Wave Height (m),Wave Period (s),Wind U (m/s),Wind V (m/s)
Date,,,,,,,,
2018-12-27 18:00:00,24.7313,-144.451,18.9648,98,0.555551,6.95097,-3.34469,1.105580
2018-12-28 00:00:00,24.4058,-142.135,19.9130,97,0.882650,8.01462,-2.13369,2.105250
2018-12-28 06:00:00,24.1036,-139.974,19.7094,96,0.803000,7.87731,-1.82490,2.417350
2018-12-28 12:00:00,23.7952,-137.767,19.3417,98,0.757344,7.73223,-1.97757,1.252120
2018-12-28 18:00:00,23.4976,-135.652,20.1010,97,0.853201,6.94640,-2.24323,0.570888


In [146]:
date = env.index.to_list() # 어떻게 하면 환경위치정보와 선박이동 데이터를 합칠 수 있을까? (선박이동 데이터의 경우에는 데이터양이 너무 작기 때문에 분류성능이 안좋음)
result_list = []

for i in date:

  try:

    df = route[(route["Date from"] <= i) & (route["Date to"] >= i)] # 환경위치정보의 인덱스는 6시간단위로 측정된 것, 따라서 이 6시간 단위의 시간이 선박루트에 있는 "Date from"과 "Date to" 사이에 있다면 해당 6시간동안 route 데이터의 항로를 사용한 것이라고 판단
    df = df.iloc[0,:].values
    result_list.append(df)

  except: # 6시간 동안 date from 과 date to 사이에 들어가지 않을경우 즉 해당시간동안 특정 항로를 통해 이동했다는 데이터가 없을 경우에는 예외처리를 하여 결측치를 삽입

    empty_list = np.empty((1,15))
    empty_list[:] = np.nan
    result_list.append(empty_list[0].tolist())

df = pd.DataFrame(result_list,columns=route.columns,index=env.index)

In [147]:
df

,Date from,Date to,days,Port from,Port to,Distance,T from,T to,Loaing Status,Mega-Ton-Mile,Cargo Carried,DFOC,Wave (per Distance),Acc. Fatigue Index,Wind (per Distance),Avg. Wave Heigh,Avg. Speed,Avg. Win Speed
Date,,,,,,,,,,,,,,,,,,
2018-01-02 12:00:00,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-02 18:00:00,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-03 00:00:00,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-03 06:00:00,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-03 12:00:00,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-27 18:00:00,2018-12-26 21:20:00,2018-12-29 13:32:00,2.7,BR0145,BR0010,19,9,16,NaN,254.266,75096,0.0,0.5,1.3,1.2,0.4,7.7,1.1
2018-12-28 00:00:00,2018-12-26 21:20:00,2018-12-29 13:32:00,2.7,BR0145,BR0010,19,9,16,NaN,254.266,75096,0.0,0.5,1.3,1.2,0.4,7.7,1.1
2018-12-28 06:00:00,2018-12-26 21:20:00,2018-12-29 13:32:00,2.7,BR0145,BR0010,19,9,16,NaN,254.266,75096,0.0,0.5,1.3,1.2,0.4,7.7,1.1


In [148]:
env = pd.concat([env,df],axis=1)

In [149]:
env.isnull().sum()

Vessel Latitude           0
Vessel Longitude          0
Vessel Speed (Kts)        0
Vessel Heading            0
Wave Height (m)           0
Wave Period (s)           0
Wind U (m/s)              0
Wind V (m/s)              0
Date from                40
Date to                  40
days                     40
Port from               323
Port to                 323
Distance                 40
 T from                 323
 T to                   323
Loaing Status          1406
Mega-Ton-Mile           323
Cargo Carried           323
DFOC                    323
Wave (per Distance)     323
Acc. Fatigue Index      323
Wind (per Distance)     323
Avg. Wave Heigh          40
Avg. Speed               40
Avg. Win Speed          323
dtype: int64

In [150]:
env.drop("Loaing Status",axis=1,inplace=True) # Loading Status는 결측값만으로 이루어져있는 column이기 떄문에 제거

# Training(Port to)


In [151]:
train_df = env[env["Port to"] != "unown"] # 목적지 항구를 아는 경우 train
val_df = env[env["Port to"] == "unown"] # 목적지 항구를 알지 못하는 경우 val 

train_df.dropna(inplace=True) # 결측치 처리
val_df.dropna(inplace=True)

date_from = val_df["Date from"].to_frame(name="Date from")
date_to = val_df["Date to"].to_frame(name="Date to")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [152]:
train_df.drop(["Port from","Date from","Date to"],axis=1,inplace=True) # 필요없는 데이터들 처리
val_df.drop(["Port from","Date from","Date to"],axis=1,inplace=True) 

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [153]:
# X,y 구분

X_train = train_df.drop("Port to",axis=1) 
y_train = train_df.loc[:,"Port to"].to_frame(name="Port to")
X_val = val_df.drop("Port to",axis=1)
y_val = val_df.loc[:,"Port to"].to_frame(name="Port from")

In [154]:
# 0~1사이 값으로 스케일링

X_train = pd.DataFrame(scaler.fit_transform(X_train),index=X_train.index,columns=X_train.columns)
y_train = pd.DataFrame(encoder.fit_transform(y_train),index=y_train.index,columns=y_train.columns)
X_val = pd.DataFrame(scaler.fit_transform(X_val),index=X_val.index,columns=X_val.columns)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [155]:
 X_train, X_test, y_train, y_test = train_test_split(X_train,y_train,test_size=0.5) # 테스트 사이즈는 오버피팅을 피하기 위해 크게 50%, 추후 하이퍼파라미터 튜닝을 통해 오버피팅을 피해야함

In [156]:
# 트레이닝

RFC = RandomForestClassifier()

RFC.fit(X_train,y_train)
pred = RFC.predict(X_test)
accuracy = accuracy_score(y_test,pred)
f1 = f1_score(y_test,pred,average="micro")
print(accuracy)
print(f1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.977961432506887
0.977961432506887


# Validation set prediction

In [157]:
val_pred = encoder.inverse_transform(RFC.predict(X_val)) 

In [158]:
port_to_result = pd.DataFrame(val_pred,index=val_df.index,columns=["Port to"])
port_to_result = pd.concat([port_to_result,date_from,date_to],axis=1)

In [159]:
result = port_to_result.groupby("Date to")["Port to"].value_counts().to_frame(name="Count") # Date to로 groupby

In [161]:
result # Port to는 결국 도착지에 도착하게 되면 Port from 이기 때문에 '전에 출발한 Port' = '어디서 온 Port'. Port to를 기준으로 하여 Port name을 예상, 동일 일자별로 다른 항구가 있을시에는 다수를 채택

Count
Date to             Port to       
2018-02-23 05:30:00 U0010        4
2018-03-17 20:24:00 U0273        4
                    U1238        1
2018-03-24 00:07:00 U0273        3
                    VE0040       2
2018-04-01 16:30:00 U0010        3
2018-04-06 21:33:00 U0010        5
2018-04-08 13:37:00 U0010        3
2018-04-11 13:09:00 U1238       10
                    U0273        2
2018-04-12 21:36:00 U0613        4
                    U0273        1
2018-04-20 20:06:00 U0010        4
2018-04-21 22:13:00 U0010        4
2018-04-28 22:25:00 U0273        7
2018-05-05 03:21:00 U1241        5
2018-06-09 20:16:00 VE0040       8
2018-06-11 17:42:00 U0010        5
2018-06-21 07:01:00 U1238       16
2018-07-22 21:21:00 U1238       55
2018-08-07 18:47:00 U0035       39
2018-08-18 16:50:00 U0991       38
                    U0035        5
2018-09-08 07:12:00 U0035       42
2018-09-28 18:32:00 U1238       62
2018-10-08 23:36:00 U0035        4
2018-10-10 09:57:00 U0991        3
2018-11-10 02:17:00 U0035       11
2018-12-22 13:39:00 BR0145       8

In [140]:
#port_to_result.to_csv("/content/drive/MyDrive/Dataset/upa/result/T-001_port_to_result.csv")